In [1]:
import pandas as pd
import re

In [2]:
def read_in_compound():
    KEGG_COMPOUND_FILE = '/vortexfs1/home/halexander/KEGG/ligand/compound/compound'
    KEGG_COMPOUND_DICT = {}
    Cnum = {}
    with open (KEGG_COMPOUND_FILE) as f:
        for i, line in enumerate(f):
    #         if i==10000:
    #             break
            if line.startswith('///'):
                for key in ['BRITE', 'DBLINKS','ATOM','BOND','BRACKET','REMARK']:
                    if key in Cnum:
                        Cnum.pop(key)
                KEGG_COMPOUND_DICT[Cnum['ENTRY']]=Cnum
                Cnum={}

            elif re.match('^[A-Z]', line):
                test=line.strip().split()
                cat = test[0]
                value = test[1:]
                if cat == 'ENTRY':
                    value=value[0]
                if cat == 'PATHWAY':
                    value=[value[0]]
                if cat == 'MODULE':
                    value=value[0]    
            else:
                test2 = line.strip().split()
                if cat == 'ENTRY':
                    value+=test2[0]
                elif cat == 'PATHWAY':
                    value+=[test2[0]]
                elif cat == 'MODULE':
                    value+=test2[0]
                else:
                    value+=test2
            Cnum[cat]=value
    return KEGG_COMPOUND_DICT
KEGG_COMPOUND_DICT=read_in_compound()        

In [3]:
KEGG_COMPOUND_DICT['C00719']

{'ENTRY': 'C00719',
 'NAME': ['Betaine;',
  'Trimethylaminoacetate;',
  'Glycine',
  'betaine;',
  'N,N,N-Trimethylglycine;',
  'Trimethylammonioacetate'],
 'FORMULA': ['C5H11NO2'],
 'EXACT_MASS': ['117.079'],
 'MOL_WEIGHT': ['117.1463'],
 'REACTION': ['R02565',
  'R02566',
  'R02821',
  'R07228',
  'R07244',
  'R08211',
  'R08212',
  'R10061',
  'R10062'],
 'PATHWAY': ['map00260', 'map01100', 'map02010'],
 'MODULE': 'M00555',
 'ENZYME': ['1.1.3.17',
  '1.2.1.8',
  '1.21.4.4',
  '2.1.1.5',
  '2.1.1.157',
  '2.1.1.161',
  '2.1.1.162']}

In [19]:
def read_in_reaction():
    KEGG_REACTION_FILE = '/vortexfs1/home/halexander/KEGG/ligand/reaction/reaction'
    KEGG_REACTION_DICT = {}
    Cnum = {}
    with open (KEGG_REACTION_FILE) as f:
        for i, line in enumerate(f):
#             if i==10000:
#                 break
            if line.startswith('///'):
                for key in ['BRITE', 'DBLINKS','ATOM','BOND','BRACKET','COMMENT']:
                    if key in Cnum:
                        Cnum.pop(key)

                KEGG_REACTION_DICT[Cnum['ENTRY']]=Cnum
                Cnum={}

            elif re.match('^[A-Z]', line):
                test=line.strip().split()
                cat = test[0]
                value = test[1:]
                if cat == 'ENTRY':
                    value=value[0]
                if cat == 'NAME':
                    value=[' '.join(value)]
                if cat == 'ORTHOLOGY':
                    value=[value[0]]
                if cat == 'DEFINITION':
                    value=' '.join(value)
                if cat == 'EQUATION':
                    value=' '.join(value)
                if cat == 'PATHWAY':
                    value=[value[0]]
                if cat == 'MODULE':
                    value=value[0]    
            else:
                test2 = line.strip().split()
                if cat == 'ENTRY':
                    value+=test2[0]
                elif cat == 'PATHWAY':
                    value+=[test2[0]]
                elif cat == 'MODULE':
                    value+=test2[0]
                elif cat == 'ORTHOLOGY':
                    value+=[test2[0]]
                else:
                    value+=test2
            Cnum[cat]=value
    return KEGG_REACTION_DICT
KEGG_REACTION_DICT=read_in_reaction()

In [119]:
KEGG_REACTION_DICT['R02566']

{'ORTHOLOGY': ['K00130', 'K14085'],
 'ENTRY': 'R02566',
 'NAME': ['p-Cumic alcohol:NADP+ oxidoreductase'],
 'DEFINITION': 'Betaine aldehyde + NADP+ + H2O <=> Betaine + NADPH + 2 H+',
 'EQUATION': 'C00576 + C00006 + C00001 <=> C00719 + C00005 + 2 C00080',
 'RCLASS': ['RC00001', 'C00005_C00006', 'RC00080', 'C00576_C00719'],
 'ENZYME': ['1.2.1.8'],
 'PATHWAY': ['rn00260', 'rn01100'],
 'MODULE': 'M00555'}

In [151]:
COMPOUND_LIST = ['C06231','C00719', 'C16432','C00334']
for c in COMPOUND_LIST:
    df=pd.DataFrame(columns=['DEFINITION', 'NAME', 'ENZYME', 'ORTHOLOGY'])
    for R in KEGG_COMPOUND_DICT[c]['REACTION']:
        KD=KEGG_REACTION_DICT[R]
        if 'NAME' in KD:
            df.loc[R,'NAME']=KD['NAME']
        if 'ENZYME' in KD:
            df.loc[R,'ENZYME']=KD['ENZYME']
    
        df.loc[R,'DEFINITION']=KD['DEFINITION']
        if 'ORTHOLOGY' in KD:
            df.loc[R,'ORTHOLOGY']=';'.join(KD['ORTHOLOGY'] )
    df.to_csv(c+'-REACTION-INFO.txt', sep='\t')

In [73]:
compound_dict = {}
with open('compound_list', 'r') as f:
    for line in f:
        ll =line.strip().split(',')
        compound_dict[ll[0]] = ll[1]
        

In [20]:
for c in compound_dict.values():
    df=pd.DataFrame(columns=['DEFINITION', 'NAME', 'ENZYME', 'ORTHOLOGY'])
    for R in KEGG_COMPOUND_DICT[c]['REACTION']:
        KD=KEGG_REACTION_DICT[R]
        if 'NAME' in KD:
            df.loc[R,'NAME']=KD['NAME']
        if 'ENZYME' in KD:
            df.loc[R,'ENZYME']=KD['ENZYME']
    
        df.loc[R,'DEFINITION']=KD['DEFINITION']
        if 'ORTHOLOGY' in KD:
            df.loc[R,'ORTHOLOGY']=';'.join(KD['ORTHOLOGY'] )
    df.to_csv(c+'-REACTION-INFO.txt', sep='\t')

In [22]:
import os
import glob


In [67]:
# Automated parsing:
automated_dict = {}
for f in glob.glob('compound_scraping/unique-kegg/automated/*'):
    kset = set()
    name = os.path.basename(f).split('-')[0]
    with open(f) as ff:
        for line in ff:
            kset.add(line.strip())
        automated_dict[name]=kset
    

In [68]:
manual_dict = {}
for f in glob.glob('compound_scraping/unique-kegg/manual/*'):
    name = os.path.basename(f).split('_')[0]
    print(name)
    kset = set()
    with open(f) as ff:
        for line in ff:
            kset.add(line.strip())
        manual_dict[name]=kset

C01104
C00794
C01083
C00064
C00089
C00719
C00116
C04022
C00334
C00245
C00213
C00392
C00148
C00025
C06231
C01004


In [101]:
for n in compound_dict.keys():
    c = compound_dict[n]
    m = manual_dict[c]
    a = automated_dict[c]
    print(','.join([n, c, ';'.join(a.difference(m))]))
#     print(len(a.intersection(m)))
#     print('\t'.join([n, c, str(len(m)), str(len(a))]))

glycine-betaine,C00719,
ectoine,C06231,
GABA,C00334,K19191
glycerol,C00116,K13700;K01108;K06131;K13704;K00968;K01059;K20983
mannitol,C00392,K03884;K00330;K03882;K00338;K00332;K00333;K00343;K00341;K00334;K03883;K00331;K00337;K00340;K00336;K00335;K02799;K03878;K00339;K00329;K03879;K03880;K17738;K03881;K00342
sacrosine,C00213,
sorbitol,C00794,K02304;K02302;K03794
sucrose,C00089,K00161;K00163;K12154;K11258;K02809;K02808;K01652;K00162;K01653;K01568;K12155
taurine,C00245,K07255;K07256
trehalose,C01083,K02817;K18478;K02818
TMAO,C01104,K05349;K01188;K05350;K00317;K13032
DMSP,C04022,
proline,C00148,K21618
glutamate,C00025,K01665;K14244;K01937;K12645;K05597;K01664;K01953;K20930;K02224;K01950;K01955;K17066;K03342;K13503;K00086;K02501;K08681;K10572;K06215;K01922;K01196;K13038;K00264;K01656;K02500;K13063;K01954;K21620;K01425;K01657;K00632;K00764;K08969;K01658;K13497;K04075;K20423;K10150;K11540;K19786;K11541;K17218;K21175;K01956;K00820;K01663;K00705;K00276;K13950;K01952;K13501;K02232;K20159;K01951
g